## Non-Negative Matrix Factorization for Exploring the European Parliament's Topic Agenda
### Assignment 2 for Machine Learning Complements class
By Alexandra de Carvalho, Luís Costa, Nuno Pedrosa

#### Importing the needed Python libraries
We will use Pandas for dataframe manipulation.

In [16]:
#!pip install gensim

In [4]:
import os
import re
import math
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

import pandas as pd

# for modeling 
from sklearn.feature_extraction import DictVectorizer
from sklearn.decomposition import NMF

# for text processing
from nltk.corpus import stopwords
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer

from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity,cosine_distances

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/luismiguel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/luismiguel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/luismiguel/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/luismiguel/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


### Static NMF

#### Importing the data

In [5]:
# expand pandas df column display width to enable easy inspection
pd.set_option('max_colwidth', 150)

# read the textfiles to a dataframe
dir_path = 'sample' # folder path
files = [] # list to store files

for path in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, path)):
        files.append(os.path.join(dir_path, path))
    else:
        subpath = os.path.join(dir_path, path)
        for path2 in os.listdir(subpath):
            if os.path.isfile(os.path.join(subpath, path2)):
                files.append(os.path.join(subpath, path2))

#### Tokenizing
To make all of the text in the speeches as comparable as possible we need to remove punctuation, capitalization, numbers, and strange characters. We also keep the term frequency on each document.

In [6]:
text_tokens = dict()
for filename in files:
    with open(filename, 'rb') as f:
        lines = f.readlines()
        text_tokens[filename] = dict()
        
        for line in lines:
            for token in re.split('\W+', str(line)):
                token = token.lower()
                if len(token) > 3 and not token.isnumeric() and not token.lower() in stopwords.words('english'):
                    text_tokens[filename][token] = text_tokens[filename].get(token, 0) + 1

#### Lemmatizing

Another important step is to lemmatize the words, that is to convert verbs and adjetives into their base form. This allows us to analyse related words as a single one and reduces the number of words in the documents matrix, reducing sparcity.

Ex: walked -> walk

In [7]:
wordnet_lemmatizer = WordNetLemmatizer()   # stored function to lemmatize each word
is_noun = lambda pos: pos[:2] == 'NN'

nouns = dict()
for filename, tokens in text_tokens.items():
    if filename not in nouns:
        nouns[filename] = dict()

    for (word, pos) in pos_tag(list(tokens.keys())):
        if is_noun(pos):
            nouns[filename][wordnet_lemmatizer.lemmatize(word)] = nouns[filename].get(wordnet_lemmatizer.lemmatize(word), 0) + text_tokens[filename][word]

#### Building the matrix A

Firstly, only with the term frequency weights.

In [8]:
dictvectorizer = DictVectorizer(sparse=False)
a = dictvectorizer.fit_transform(list(nouns.values()))

Building the list of all tokens (all columns of A, in order).

In [9]:
token_list = dictvectorizer.get_feature_names()

/Users/luismiguel/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Now calculating updating to TF-IDF weights

In [10]:
for column_idx in range(len(token_list)):
    idf = math.log(len(a[:, column_idx])/len([x for x in a[:, column_idx] if x != 0]), 10)

    for element_idx in range(len(files)):
        if a[element_idx,column_idx] != 0:
            a[element_idx,column_idx] = (math.log(a[element_idx,column_idx], 10) + 1) * idf

#### Finding the best value for K : TC-W2V

We used the same number of terms per topic *t* as the paper.

In [11]:
# number of terms per topic
t = 10

We run the NMF with a number of topics that ranges between 10 and 26 in order to find the ammount of topics that gives us a model with the biggest coherence. The coherence of each topic is measured using the cosine similarity. The coherence of a model is the mean coherence of the topics of the model.

In [12]:
max_model_coherence = 0
res_k = 0

for k in range(10,26):

    nmf_model = NMF(k) 
    nmf_model.fit_transform(a)

    vocabulary = [[token_list[x[1]] for x in sorted(zip(topic,range(len(topic))), reverse = True)[:t]] for topic in nmf_model.components_]
    model = Word2Vec(sentences = vocabulary, vector_size = 200, window = 5, hs = 1, negative = 0, min_count = 1)
    
    # calculating individual topic coherence scores for each topic
    model_score = []
    for topic in vocabulary:
        topic_score = []
        for w1 in topic:
            for w2 in topic:
                if w2 > w1:
                    word_score = cosine_similarity(model.wv[w2].reshape(1,-1),model.wv[w1].reshape(1,-1))[0]
                    topic_score.append(word_score[0])
        
        topic_score = sum(topic_score)/len(topic_score) # mean of each word pair similarity in the topic
        model_score.append(topic_score)

    model_coherence = sum(model_score)/len(model_score) # mean of topic coherence in the model
    print("k = ",k, ". Model coherence:", model_coherence)

    # used in order to choose the number of topics that has the biggest coherence
    if model_coherence > max_model_coherence:
        max_model_coherence = model_coherence
        res_k = k

/Users/luismiguel/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


k =  10 . Model coherence: 0.0010334041642232083


/Users/luismiguel/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


k =  11 . Model coherence: 0.0011395746625658836


/Users/luismiguel/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


...

k =  23 . Model coherence: 0.002602588914442753


/Users/luismiguel/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


k =  24 . Model coherence: 0.0034506020235346145


/Users/luismiguel/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


k =  25 . Model coherence: -0.0014814236255155672


#### NMF

The number of topics that gives us the biggest coherence is 22, so that is the number of topics that we will use in our definitive NMF. 

In [13]:
nmf_model = NMF(res_k) 
w = nmf_model.fit_transform(a)

/Users/luismiguel/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(


#### Results

For each topic, find the t higher weights' index and find the correpondent token (same index) in the token list. These are the descriptors of each topic.

In [14]:
for i, topic in enumerate(nmf_model.components_):
    print("Topic", i, ":",[token_list[x[1]] for x in sorted(zip(topic,range(len(topic))), reverse = True)[:t]])

Topic 0 : ['government', 'education', 'minister', 'health', 'child', 'school', 'department', 'council', 'proposal', 'report']
Topic 1 : ['player', 'club', 'game', 'football', 'team', 'season', 'chelsea', 'manager', 'champion', 'match']
Topic 2 : ['lord', 'court', 'right', 'law', 'trial', 'case', 'detainee', 'police', 'lawyer', 'detention']
Topic 3 : ['music', 'band', 'song', 'rock', 'singer', 'artist', 'album', 'single', 'record', 'chart']
Topic 4 : ['forsyth', 'frederick', 'terrorist', 'internment', 'forsythe', 'totalitarianism', 'qaeda', 'fundamentalism', 'churchill', 'liberty']
Topic 5 : ['growth', 'economy', 'market', 'price', 'rate', 'rise', 'bank', 'analyst', 'dollar', 'profit']
Topic 6 : ['angel', 'rhapsody', 'bland', 'guy', 'pulp', 'cheesy', 'brit', 'scissor', 'joke', 'listener']
Topic 7 : ['sub', 'ball', 'minute', 'cech', 'duff', 'goal', 'header', 'kezman', 'robben', 'shot']
Topic 8 : ['virus', 'program', 'mail', 'security', 'software', 'computer', 'attack', 'user', 'machine',

We can then observe the documents with bigger weights for each topic. Because the files names already tag the contained speech by topic, we can infer the validity of the model built.

In [15]:
for i in range(res_k):
    print("Topic", i, ":",[files[x[1]].split('/')[-1] for x in sorted(zip(w[:,i],range(len(w[:,i]))), reverse = True)[:t]])

Topic 0 : ['entertainment_131.txt', 'politics_080.txt', 'politics_244.txt', 'politics_055.txt', 'politics_056.txt', 'politics_246.txt', 'business241.txt', 'politics_020.txt', 'tech_399.txt', 'politics_127.txt']
Topic 1 : ['football_207.txt', 'football_087.txt', 'football_088.txt', 'football_086.txt', 'football_223.txt', 'football_202.txt', 'football_085.txt', 'football_221.txt', 'football_141.txt', 'football_124.txt']
Topic 2 : ['politics_224.txt', 'politics_040.txt', 'politics_221.txt', 'politics_137.txt', 'politics_192.txt', 'politics_160.txt', 'politics_205.txt', 'politics_168.txt', 'politics_066.txt', 'politics_134.txt']
Topic 3 : ['entertainment_262.txt', 'entertainment_229.txt', 'entertainment_244.txt', 'entertainment_263.txt', 'entertainment_142.txt', 'entertainment_131.txt', 'entertainment_133.txt', 'entertainment_145.txt', 'entertainment_236.txt', 'entertainment_153.txt']
Topic 4 : ['politics_290.txt', 'politics_052.txt', 'politics_160.txt', 'politics_263.txt', 'politics_189.t

As we can see, theres is definitively a match between the terms in each topic and the name of the speechs from which they were recovered. This results are aligned with what was expected and as such verify that the model built is a valid one.